In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [55]:
import torch
import torchaudio as ta
import torchaudio.functional as taf
import torchaudio.transforms as tat
from torchvision import transforms

print(torch.__version__)        
print(ta.__version__)

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Audio, display

import pandas as pd
import os
import pprint

from typing import *
import itertools
from collections import Counter

import numpy as np
from datetime import datetime
from lark.config import Config
from lark.learner import Learner
from lark.ops import Sig2Spec, MixedSig2Spec, f1
from lark.data import *


from tqdm.notebook import tqdm

1.8.1
0.8.0a0+e4e171a


In [56]:
torch.cuda.set_device(0)
torch.cuda.current_device()

0

In [57]:
# sites = ['SSW']
sites = None
cfg = Config(sites)

In [58]:
df_ss = pd.read_csv(f"{cfg.data_dir}/train_soundscape_labels.csv")
df_ss['audio_id'] = df_ss['audio_id'].astype(str)
df_ss = df_ss.sort_values(by=['audio_id', 'site', 'seconds'])
if cfg.sites is not None:
    df_ss = df_ss[df_ss['site'].isin(cfg.sites)].reset_index(drop=True)
indices = {b: cfg.labels.index(b) for b in cfg.labels}

def row_label(row):
    label = torch.zeros(len(cfg.labels))
    if row.birds != 'nocall':
        birds = row.birds.split(' ')
        ixs = [indices[b] for b in birds]
        label[ixs] = 1.0
    return label

df_ss['label'] = df_ss.apply(lambda row: row_label(row), axis=1)
ys = torch.Tensor(np.stack(df_ss.label))

In [66]:
df_ss

,row_id,site,audio_id,seconds,birds,label
1440,10534_SSW_5,SSW,10534,5,nocall,"[tensor(0.), tensor(0.), tensor(0.), tensor(0...."
1441,10534_SSW_10,SSW,10534,10,swaspa,"[tensor(0.), tensor(0.), tensor(0.), tensor(0...."
1442,10534_SSW_15,SSW,10534,15,nocall,"[tensor(0.), tensor(0.), tensor(0.), tensor(0...."
1443,10534_SSW_20,SSW,10534,20,nocall,"[tensor(0.), tensor(0.), tensor(0.), tensor(0...."
1444,10534_SSW_25,SSW,10534,25,swaspa,"[tensor(0.), tensor(0.), tensor(0.), tensor(0...."
...,...,...,...,...,...,...
235,7954_COR_580,COR,7954,580,nocall,"[tensor(0.), tensor(0.), tensor(0.), tensor(0...."
236,7954_COR_585,COR,7954,585,nocall,"[tensor(0.), tensor(0.), tensor(0.), tensor(0...."
237,7954_COR_590,COR,7954,590,nocall,"[tensor(0.), tensor(0.), tensor(0.), tensor(0...."
238,7954_COR_595,COR,7954,595,nocall,"[tensor(0.), tensor(0.), tensor(0.), tensor(0...."


In [59]:
# model_dict = [
#     {
#         'repo': 'pytorch/vision:v0.9.0',
#         'model': 'resnet18',
#         'checkpoint': 'resnet18-half-frozen-bag-0-20210510-042633'
#     },
#     {
#         'repo': 'pytorch/vision:v0.9.0',
#         'model': 'resnet18',
#         'checkpoint': 'resnet18-half-frozen-bag-1-20210510-043006'
#     },
# ]

In [60]:
model_dict = [
    {
        'repo': 'zhanghang1989/ResNeSt',
        'model': 'resnest50',
        'checkpoint': 'resnest50-2ndary-labels-full-20210506-221103-latest'
    },
    {
        'repo': 'zhanghang1989/ResNeSt',
        'model': 'resnest50',
        'checkpoint': 'resnest50-full-training-20210505-132542-latest'
    },
    
#     {
#         'repo': 'zhanghang1989/ResNeSt',
#         'model': 'resnest50',
#         'checkpoint': 'resnest50-2ndary-labels-20210506-160059-latest'
#     },
#     {
#         'repo': 'pytorch/vision:v0.9.0',
#         'model': 'resnet18',
#         'checkpoint': 'resnet18-half-frozen-20210507-125647-latest'
#     },
]

In [61]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [62]:
def init_model(repo, model, checkpoint):
    main_model = torch.hub.load(repo, model, pretrained=False)
    prep = MixedSig2Spec(cfg)
    n_features = main_model.fc.in_features
    main_model.fc = torch.nn.Linear(in_features=n_features, out_features=cfg.n_labels, bias=True)
    model = torch.nn.Sequential(prep, main_model)
    cp = torch.load(f"checkpoints/{checkpoint}.pt", map_location=torch.device(device))
    model.load_state_dict(cp['model_state_dict'])
    model = model.to(device).eval()
    return model

In [63]:
def list_files(path):
    try:
        return sorted([os.path.join(path, f) for f in os.listdir(path) if f.rsplit('.', 1)[-1] in ['ogg']])
    except:
        return []
test_audio = list_files('data/birdclef-2021/test_soundscapes')
if len(test_audio) == 0:
    test_audio = list_files('data/birdclef-2021/train_soundscapes')

n_files = len(test_audio)
print('{} FILES IN TEST SET.'.format(len(test_audio)))

20 FILES IN TEST SET.


In [64]:
ps = torch.zeros((n_files, 120, cfg.n_labels))
for m in model_dict:
    model = init_model(m['repo'], m['model'], m['checkpoint'])
    with torch.no_grad():
        for i, path in enumerate(tqdm(test_audio)):
            sig, sr = ta.load(filepath=path)
            sig = sig.to(device).reshape(-1, 1, cfg.sr * cfg.valid_duration)
            probs = model(sig).sigmoid().cpu()
            del sig
            ps[i] += probs
ps = ps.reshape(-1, cfg.n_labels)
ps /= len(model_dict)

Using cache found in /home/koen/.cache/torch/hub/zhanghang1989_ResNeSt_master


  0%|          | 0/20 [00:00<?, ?it/s]

Using cache found in /home/koen/.cache/torch/hub/zhanghang1989_ResNeSt_master


  0%|          | 0/20 [00:00<?, ?it/s]

In [65]:
ts = np.arange(0.0, 1.1, 0.1)
rs = [f1(ys, ps, t) for t in ts]
df = pd.DataFrame(rs)
df

,thresh,tp,tn,fp,fn,f1
0,0.0,1183,0,951617,0,0.002480
1,0.1,199,949080,2537,984,0.101557
2,0.2,139,950704,913,1044,0.124385
3,0.3,108,951243,374,1075,0.129730
4,0.4,80,951525,92,1103,0.118081
5,0.5,44,951605,12,1139,0.071025
6,0.6,25,951614,3,1158,0.041288
7,0.7,20,951616,1,1163,0.033223
8,0.8,11,951616,1,1172,0.018410
9,0.9,7,951616,1,1176,0.011755
